<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
تحلیل آماری داده‌ها
</font>
</h1>

<p dir="rtl" style="direction: rtl; text-align: justify; line-height: 2; font-family: Vazir, sans-serif; font-size: medium;">
  <strong>خروجی نهایی</strong> باید شامل ۳۲ مقدار به شکل یک فایل 
  <code>submission.csv</code> با ۶ رقم اعشار باشد.
  در هر بخش مقدار(های) لازم را در متغیرهای 
  <code>value_01</code> تا <code>value_32</code> قرار دهید. 
  در انتها این مقادیر استخراج و ذخیره می‌شوند.
</p>

<blockquote dir="rtl" style="direction: rtl; text-align: justify; font-family: Vazir, sans-serif; font-size: medium; line-height: 2;">
  <strong>یادآوری:</strong> برای تمام محاسبات «وینسور» (Winsorization) روی هر سنسور به صورت مستقل انجام می‌دهیم:  
  برای هر سنسور صدک‌های ۱٪ و ۹۹٪ را با 
  <code>interpolation='linear'</code> محاسبه کرده و سپس با 
  <code>clip</code> مقادیر را به این بازه محدود می‌کنیم.  
  مگر جایی که صراحتاً گفته شده «خام/بدون وینسور». همچنین برای 
  <code>std</code> از نمونه‌ای (<code>ddof=1</code>) استفاده کنید.
</blockquote>


<h1 dir="rtl" style="direction: rtl; text-align: right; font-family: Vazir, sans-serif; font-size: x-large; line-height: 2;">
  <font face="vazir" color="#cc0000ff">

  بارگذاری داده و آماده‌سازی
     </font>

</h1>

<ul dir="rtl" style="direction: rtl; text-align: justify; font-family: Vazir, sans-serif; font-size: medium; line-height: 2;">
  <li>خواندن <code>sample.csv</code></li>
  <li>تبدیل <code>timestamp</code> به نوع زمانی</li>
  <li>تعریف تابع وینسور و نگهداشت نسخهٔ وینسورشدهٔ هر سنسور</li>
</ul>


In [1]:
import os
import joblib
import zipfile
import itertools

import numpy as np
import pandas as pd

In [2]:
SENSORS = ['Temperature_C','Pressure_kPa','VibAccel_m_s2','VibVelocity_mm_s']

In [3]:
csv_path = os.path.join(os.getcwd(), "statistical_analysis_data.csv")
df = pd.read_csv(csv_path)
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

df

,timestamp,Temperature_C,Pressure_kPa,VibVelocity_mm_s,VibAccel_m_s2,FlowRate_L_min,Current_A,Voltage_V,Humidity_pct,pH_units,OilLevel_cm,Power_kW,Speed_RPM,Torque_Nm,BearingTemp_C,VibDisp_mm,status,RUL_seconds
0,2025-01-01 00:00:00,56.341128,353.860999,9.549631,25.039562,25.877314,4.745597,228.807642,57.496418,7.359568,9.825698,51.558428,1518.527838,88.886736,65.128807,0.468871,normal,49999
1,2025-01-01 00:00:01,54.638017,353.860999,9.549631,25.039562,25.877314,5.393026,228.807642,57.496418,7.359568,9.825698,46.206392,1518.527838,88.886736,65.128807,0.468871,normal,49998
2,2025-01-01 00:00:02,56.771420,336.497724,9.549631,25.039562,25.877314,5.279771,228.733980,57.496418,7.359568,9.825698,50.746547,1605.763781,88.886736,65.128807,0.468871,normal,49997
3,2025-01-01 00:00:03,NaN,336.497724,9.549631,25.039562,25.877314,5.195886,228.733980,57.496418,7.359568,9.825698,47.626058,1605.763781,88.886736,65.128807,0.468871,normal,49996
4,2025-01-01 00:00:04,54.413107,344.572344,9.549631,25.039562,25.877314,4.688596,229.254431,57.496418,7.359568,9.825698,50.102606,1314.972873,88.886736,65.128807,0.468871,normal,49995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2025-01-01 00:16:35,64.488247,374.465002,12.650834,32.088326,30.419641,5.710645,235.083195,69.897266,7.791415,12.410745,61.557701,1738.848648,127.011607,91.120985,0.695083,equipment_fault,49004
996,2025-01-01 00:16:36,70.109263,414.289912,12.650834,32.088326,30.419641,6.459018,232.957583,69.897266,7.791415,12.410745,64.791135,1806.979533,127.011607,91.120985,0.695083,normal,49003
997,2025-01-01 00:16:37,66.994070,414.289912,12.650834,32.088326,30.419641,6.028794,232.957583,69.897266,7.791415,12.410745,61.004127,1806.979533,127.011607,91.120985,0.695083,normal,49002
998,2025-01-01 00:16:38,63.729890,416.268513,12.650834,32.088326,30.419641,5.896236,234.235203,69.897266,7.791415,12.410745,62.326512,1833.740266,127.011607,91.120985,0.695083,normal,49001


In [4]:
def winsorize(s: pd.Series) -> pd.Series:
    q1 = s.quantile(0.01, interpolation='linear')
    q99 = s.quantile(0.99, interpolation='linear')
    return s.clip(lower=q1, upper=q99)

wdf = df.copy()
for c in SENSORS:
    wdf[c] = winsorize(wdf[c])

wdf

,timestamp,Temperature_C,Pressure_kPa,VibVelocity_mm_s,VibAccel_m_s2,FlowRate_L_min,Current_A,Voltage_V,Humidity_pct,pH_units,OilLevel_cm,Power_kW,Speed_RPM,Torque_Nm,BearingTemp_C,VibDisp_mm,status,RUL_seconds
0,2025-01-01 00:00:00,56.341128,353.860999,9.549631,25.039562,25.877314,4.745597,228.807642,57.496418,7.359568,9.825698,51.558428,1518.527838,88.886736,65.128807,0.468871,normal,49999
1,2025-01-01 00:00:01,54.638017,353.860999,9.549631,25.039562,25.877314,5.393026,228.807642,57.496418,7.359568,9.825698,46.206392,1518.527838,88.886736,65.128807,0.468871,normal,49998
2,2025-01-01 00:00:02,56.771420,336.497724,9.549631,25.039562,25.877314,5.279771,228.733980,57.496418,7.359568,9.825698,50.746547,1605.763781,88.886736,65.128807,0.468871,normal,49997
3,2025-01-01 00:00:03,NaN,336.497724,9.549631,25.039562,25.877314,5.195886,228.733980,57.496418,7.359568,9.825698,47.626058,1605.763781,88.886736,65.128807,0.468871,normal,49996
4,2025-01-01 00:00:04,54.413107,344.572344,9.549631,25.039562,25.877314,4.688596,229.254431,57.496418,7.359568,9.825698,50.102606,1314.972873,88.886736,65.128807,0.468871,normal,49995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2025-01-01 00:16:35,64.488247,374.465002,12.650834,32.088326,30.419641,5.710645,235.083195,69.897266,7.791415,12.410745,61.557701,1738.848648,127.011607,91.120985,0.695083,equipment_fault,49004
996,2025-01-01 00:16:36,69.628905,414.289912,12.650834,32.088326,30.419641,6.459018,232.957583,69.897266,7.791415,12.410745,64.791135,1806.979533,127.011607,91.120985,0.695083,normal,49003
997,2025-01-01 00:16:37,66.994070,414.289912,12.650834,32.088326,30.419641,6.028794,232.957583,69.897266,7.791415,12.410745,61.004127,1806.979533,127.011607,91.120985,0.695083,normal,49002
998,2025-01-01 00:16:38,63.729890,416.268513,12.650834,32.088326,30.419641,5.896236,234.235203,69.897266,7.791415,12.410745,62.326512,1833.740266,127.011607,91.120985,0.695083,normal,49001


<h1 dir="rtl" style="direction: rtl; text-align: right; font-family: Vazir, sans-serif; font-size: x-large; line-height: 2;">
  <font face="vazir" color="#cc0000ff">
     مقداردهی اولیهٔ متغیرهای پاسخ
  </font>
</h1>

<p dir="rtl" style="direction: rtl; text-align: justify; font-family: Vazir, sans-serif; font-size: medium; line-height: 2;">
  در هر بخش متغیرهای <code>value_XX</code> را <strong>جایگزین/محاسبه</strong> کنید.
</p>


In [5]:
values = []

value_01 = None
value_02 = None
value_03 = None
value_04 = None
value_05 = None
value_06 = None
value_07 = None
value_08 = None
value_09 = None
value_10 = None
value_11 = None
value_12 = None
value_13 = None
value_14 = None
value_15 = None
value_16 = None
value_17 = None
value_18 = None
value_19 = None
value_20 = None
value_21 = None
value_22 = None
value_23 = None
value_24 = None
value_25 = None
value_26 = None
value_27 = None
value_28 = None
value_29 = None
value_30 = None
value_31 = None
value_32 = None

<h1 dir="rtl" style="direction: rtl; text-align: right; font-family: Vazir, sans-serif; font-size: x-large; line-height: 2;">
  <font face="vazir" color="#cc0000ff">
    میانگین و انحراف معیار (وینسورشده)
  </font>
</h1>

<ul dir="rtl" style="direction: rtl; text-align: justify; font-family: Vazir, sans-serif; font-size: medium; line-height: 2;">
  <li>مقادیر <strong>۱–۴</strong>: میانگین هر سنسور (به ترتیب: دما، فشار، شتاب ارتعاش، سرعت ارتعاش)</li>
  <li>مقادیر <strong>۵–۸</strong>: انحراف معیار نمونه‌ای (<code>ddof=1</code>) به همان ترتیب</li>
</ul>


In [6]:
for col in SENSORS:
    values.append(wdf[col].mean())

for col in SENSORS:
    values.append(wdf[col].std(ddof=1))

<h1 dir="rtl" style="direction: rtl; text-align: right; font-family: Vazir, sans-serif; font-size: x-large; line-height: 2;">
  <font face="vazir" color="#cc0000ff">
     کمینه و بیشینهٔ خام (بدون وینسور)
  </font>
</h1>

<ul dir="rtl" style="direction: rtl; text-align: justify; font-family: Vazir, sans-serif; font-size: medium; line-height: 2;">
  <li>مقادیر <strong>۹–۱۶</strong>: به ترتیب <code>(min, max)</code> برای هر سنسور (دما، فشار، شتاب، سرعت)</li>
</ul>


In [7]:
for col in SENSORS:
    values.append(df[col].min())
    values.append(df[col].max())

<h1 dir="rtl" style="direction: rtl; text-align: right; font-family: Vazir, sans-serif; font-size: x-large; line-height: 2;">
  <font face="vazir" color="#cc0000ff">
    همبستگی پیرسون (وینسورشده، pairwise dropna)
  </font>
</h1>

<p dir="rtl" style="direction: rtl; text-align: justify; font-family: Vazir, sans-serif; font-size: medium; line-height: 2;">
  مقادیر <strong>۱۷–۲۲</strong>: به ترتیب جفت‌ها
</p>

<ol dir="rtl" style="direction: rtl; text-align: justify; font-family: Vazir, sans-serif; font-size: medium; line-height: 2;">
  <li><code>(Temperature_C, Pressure_kPa)</code></li>
  <li><code>(Temperature_C, VibAccel_m_s2)</code></li>
  <li><code>(Temperature_C, VibVelocity_mm_s)</code></li>
  <li><code>(Pressure_kPa, VibAccel_m_s2)</code></li>
  <li><code>(Pressure_kPa, VibVelocity_mm_s)</code></li>
  <li><code>(VibAccel_m_s2, VibVelocity_mm_s)</code></li>
</ol>


In [8]:
combinations = list(itertools.combinations(SENSORS, 2))
for cols in combinations:
    print(cols)
    cdf = df.dropna(subset=list(cols))
    corr = wdf[cols[0]].corr(wdf[cols[1]])
    values.append(corr)

('Temperature_C', 'Pressure_kPa')
('Temperature_C', 'VibAccel_m_s2')
('Temperature_C', 'VibVelocity_mm_s')
('Pressure_kPa', 'VibAccel_m_s2')
('Pressure_kPa', 'VibVelocity_mm_s')
('VibAccel_m_s2', 'VibVelocity_mm_s')


<h1 dir="rtl" style="direction: rtl; text-align: right; font-family: Vazir, sans-serif; font-size: x-large; line-height: 2;">
  <font face="vazir" color="#cc0000ff">
    میانهٔ هر سنسور (وینسورشده)
  </font>
</h1>

<ul dir="rtl" style="direction: rtl; text-align: justify; font-family: Vazir, sans-serif; font-size: medium; line-height: 2;">
  <li>مقادیر <strong>۲۳–۲۶</strong>: میانهٔ سنسورها به ترتیب استاندارد</li>
</ul>


In [9]:
for col in SENSORS:
    values.append(wdf[col].median())

<h1 dir="rtl" style="direction: rtl; text-align: right; font-family: Vazir, sans-serif; font-size: x-large; line-height: 2;">
  <font face="vazir" color="#cc0000ff">
    خودهمبستگی با وقفهٔ ۱ (lag=1) روی دادهٔ وینسورشده
  </font>
</h1>

<ul dir="rtl" style="direction: rtl; text-align: justify; font-family: Vazir, sans-serif; font-size: medium; line-height: 2;">
  <li>مقادیر <strong>۲۷–۳۰</strong>: به ترتیب برای سنسورها</li>
</ul>


In [10]:
for col in SENSORS:
    autocorr_lag1 = wdf[col].autocorr(lag=1)
    values.append(autocorr_lag1)

<h1 dir="rtl" style="direction: rtl; text-align: right; font-family: Vazir, sans-serif; font-size: x-large; line-height: 2;">
  <font face="vazir" color="#cc0000ff">
   نرخ NaN و بازهٔ نمونه‌برداری
  </font>
</h1>

<ul dir="rtl" style="direction: rtl; text-align: justify; font-family: Vazir, sans-serif; font-size: medium; line-height: 2;">
  <li>مقدار <strong>۳۱</strong>: سهم ردیف‌هایی که <strong>حداقل یکی</strong> از چهار سنسورشان <code>NaN</code> دارد (روی دادهٔ خام)</li>
  <li>مقدار <strong>۳۲</strong>: میانهٔ فاصلهٔ نمونه‌برداری (ثانیه) پس از مرتب‌سازی صعودی زمان‌ها</li>
</ul>


In [11]:
n_rows = len(wdf)
n_with_nan = wdf[SENSORS].isna().any(axis=1).sum()
share_nan = n_with_nan / n_rows
values.append(share_nan)

deltas = df.timestamp.diff().dt.total_seconds().dropna()
median_delta = deltas.median()
values.append(median_delta)

share_nan, median_delta

(np.float64(0.088), np.float64(1.0))

In [12]:
assert len(values) == 32, "wrong values len"



<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) در غیر این صورت، در پایان مسابقه نمره شما به صفر تغییر خواهد کرد.
    <br>
    همچنین اگر از کولب برای اجرای این فایل نوت‌بوک استفاده می‌کنید، قبل از ارسال فایل <code>result.zip</code>، آخرین نسخه‌ی نوت‌بوک خود را دانلود کرده و داخل فایل ارسالی قرار دهید.
</font>

In [13]:
if not os.path.exists(os.path.join(os.getcwd(), 'notebook.ipynb')):
    %notebook -e notebook.ipynb

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

answers = values
# answers = [globals()[f'value_{i:02d}'] for i in range(1, 33)]
s = pd.Series(answers, dtype='float64', name="prediction")
s.to_csv("submission.csv", index=False, float_format="%.6f")

file_names = ['notebook.ipynb', 'submission.csv']
compress(file_names)

File Paths:
['notebook.ipynb', 'submission.csv']
